In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from fluid import Fluid


In [6]:
co2 = Fluid('co2')


['T', 'P'] [300, 100000.0]
1.773026407279758 kilogram / meter ** 3
